# Extracting multimodal data features

`siibra` provides access to data features of different modalities. The features and their query functions are bundled in the module `siibra.features`. 

In [ ]:
import siibra
from packaging.version import Version
assert Version(siibra.__version__) >= Version('1.0a08')

We can choose different types of features from this module. The feature types are organized in a hierarchy under the most abstract type `siibra.features.Feature`. All other feature types are subclasses of it. The current hierarchy can be obtained by

In [26]:
siibra.features.render_ascii_tree("Feature")

Feature
├── CompoundFeature
├── Tabular
│   ├── CorticalProfile
│   │   ├── BigBrainIntensityProfile
│   │   ├── CellDensityProfile
│   │   └── ReceptorDensityProfile
│   ├── GeneExpressions
│   │   ├── ProxyFeature
│   │   └── ProxyFeature
│   ├── LayerwiseBigBrainIntensities
│   ├── LayerwiseCellDensity
│   ├── ReceptorDensityFingerprint
│   └── RegionalTimeseriesActivity
│       └── RegionalBOLD
├── RegionalConnectivity
│   ├── FunctionalConnectivity
│   ├── AnatomoFunctionalConnectivity
│   ├── StreamlineCounts
│   ├── StreamlineLengths
│   └── TracingConnectivity
├── Image
│   ├── CellBodyStainedVolumeOfInterest
│   ├── BlockfaceVolumeOfInterest
│   ├── DTIVolumeOfInterest
│   ├── PLIVolumeOfInterest
│   ├── MRIVolumeOfInterest
│   ├── XPCTVolumeOfInterest
│   ├── LSFMVolumeOfInterest
│   └── CellbodyStainedSection
└── EbrainsDataFeature


### Example: Densities of neurotransmitter receptors

Features can be queried for brain regions, parcellations and location objects (such as volumes of interest in a refrence space) with `siibra.features.get()`, which accepts a query object and a feature type. It will query all subclasses of the given feature type, if any. Here is a simple example for getting a receptor density fingerprint in region V1. The data is of tabular type, provided as a pandas dataframe.

In [ ]:
v1 = siibra.get_region(parcellation='julich 2.9', region='v1')

In [ ]:
features = siibra.features.get(
    v1, siibra.features.molecular.ReceptorDensityFingerprint
)
print(features[0].name)
features[0].data.T  # we transpose the table for display

Most features provide a plot method for easy inspection.

In [ ]:
features[0].plot()

Typically, not all regions have all data modalities linked. Neurotransmitter density fingerprints are currently available for part of the cytoarchitectonic brain regions in human. We can see this by querying with the parcellation, which represents the root region and thus provides all features linked to any Julich-Brain regions.

Here we just print the set of unique region names for which receptor densities are linked.

In [ ]:
features = siibra.features.get(
    siibra.parcellations.get('julich'),
    siibra.features.molecular.ReceptorDensityFingerprint
)
print(
    "Regions with receptor features:\n - "
    + "\n - ".join(
        {r.name for f in features for r in f.anchor.regions}
    )
)

### Cortical cell densities

The cellular level contains layer-specific cell distributions extracted from different regions of BigBrain.

In [ ]:
features = siibra.features.get(v1, siibra.features.cellular.LayerwiseCellDensity)
print(
    f"{features[0].modality}\n\n"
    f"{features[0].description}"
)
fig = features[0].plot()

We can retrieve a similar feature representing staining intensities in BigBrain - `siibra.features.cellular.LayerwiseBigBrainIntensities`. This uses cortical staining profiles computed by Konrad Wagstyl. At first call, the query will take a bit to retrieve the profiles.

In [ ]:
features = siibra.features.get(v1, siibra.features.cellular.LayerwiseBigBrainIntensities)
print(
    f"{features[0].modality}\n\n"
    f"{features[0].description}"
)
fig = features[0].plot()

### Gene Expressions from the Allen Atlas 

`siibra` also implements a live queryto gene expression data from the Allen atlas to extract regional gene expression levels. Gene expressions are linked to atlas regions by coordinates of their probes in MNI space. When called with a brain region, `siibra.features.get` will generate a mask of this region in MNI space to filter the probes. It provides the regional expression levels as tabular data, together with the MNI coordinates.

In [ ]:
features = siibra.features.get(
    v1, siibra.features.molecular.GeneExpressions, gene=["TAC1", "MAOA", "GABARAPL2"]
)
fig = features[0].plot(title=f'Gene Expressions in {v1}')
features[0].data

We can plot the MNI coordinates to localize the measures and verify they are located in V1.

In [ ]:
from nilearn import plotting
mask = v1.get_regional_map("mni152")
display = plotting.plot_glass_brain(mask.fetch(), cmap='viridis')
locations = features[0].anchor.location
display.add_markers(locations.as_list(), marker_size=2) 

### Connectivity matrices

`siibra` provides connectivity matrices with parcellation averaged structural and functional measurments for different subjects of different cohorts. Here we request streamline counts for Julich Brain. Since the datasets comprise hundreds of connectivity matrices for the same cohort and modality, they are grouped into "compounds".

In [ ]:
features = siibra.features.get(
    siibra.parcellations.get('julich 2.9'),
    siibra.features.connectivity.StreamlineCounts
)

Let's check the cohort and indices of individual feature elements for the first compound. The elements are indexed by subject id.

In [ ]:
print(
    f"{features[0].cohort}\n\n"
    "Indices: "
    f"{', '.join(features[0].indices)}"
)

We can retrieve the matrix of a particular subject using the `get_element()` method of the compound. It includes the actual data as a pandas dataframe, with the notable property that the row and column indices are valid region objects for further reference. This implies in particular, that we can directly associate each measure with the corresponding information in the parcellation, and with a mask of a parcellation map.

In [ ]:
sc_hcp_000 = features[0].get_element('000')
sc_hcp_000.data

As an example, we retrieve the centroids in MNI152 space and plot the connnectivity graph in 3D:

In [ ]:
node_coords = sc_hcp_000.compute_centroids(space='mni152')
plotting.view_connectome(
    adjacency_matrix=sc_hcp_000.data,
    node_coords=node_coords,
    edge_threshold="99%",
    node_size=3, colorbar=False,
    edge_cmap="bwr"
)